In [10]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [11]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [12]:
from __future__ import print_function
from numpy import *
from scipy import *
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
import scipy as sp
import warnings
warnings.filterwarnings('ignore')

import seawater
import datetime as dt
""
from salishsea_tools import (
    nc_tools,
    viz_tools,
    geo_tools,
    tidetools
)

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.patches as patches
plt.style.use('seaborn-whitegrid')
import netCDF4 as nc
import time
import cmocean as cm
import glob
import sys
sys.path.append('/data/tjarniko/mocsy')
sys.path.append('/data/tjarniko/MEOPAR/at3/notebooks/carbon_dev/CCCmaDEV/CCCma_src')
import mocsy
import CCCma
import CCCma_stations as cs
from matplotlib import reload
import pickle
import arrow
import gsw
%matplotlib inline

import xarray as xr
from datetime import datetime
from salishsea_tools import grid_tools, viz_tools




In [13]:
t_nc = nc.Dataset('/data/tjarniko/results/BASERUN_EXP/MAIN/OmA_pH_calculated/LA_OmA_pH_20151018.nc')
print(t_nc['model_output']['OmAr'][:,250,250])

t_nc = nc.Dataset('/data/tjarniko/results/BASERUN_EXP/MAIN/OmA_ph_calculated_depth0//LA_OmA_pH_20151018.nc')
print(t_nc['model_output']['OmAr'][:,250,250])


[1.2951742e+00 1.2868067e+00 1.2805640e+00 1.2762913e+00 1.2726389e+00
 1.2692630e+00 1.2665161e+00 1.2640535e+00 1.2616900e+00 1.2589580e+00
 1.2563775e+00 1.2538857e+00 1.2510839e+00 1.2484863e+00 1.2452995e+00
 1.2424107e+00 1.2392055e+00 1.2356168e+00 1.2320641e+00 1.2277805e+00
 1.2229658e+00 1.2129197e+00 1.1913049e+00 1.1528704e+00 1.1026338e+00
 1.0119646e+00 9.7146761e-01 9.5515347e-01 1.0000000e+20 1.0000000e+20
 1.0000000e+20 1.0000000e+20 1.0000000e+20 1.0000000e+20 1.0000000e+20
 1.0000000e+20 1.0000000e+20 1.0000000e+20 1.0000000e+20 1.0000000e+20]
[1.2953001e+00 1.2871827e+00 1.2811884e+00 1.2771633e+00 1.2737573e+00
 1.2706273e+00 1.2681255e+00 1.2659078e+00 1.2637887e+00 1.2612997e+00
 1.2589619e+00 1.2567127e+00 1.2541530e+00 1.2517991e+00 1.2488577e+00
 1.2462218e+00 1.2432827e+00 1.2399863e+00 1.2367803e+00 1.2329439e+00
 1.2287652e+00 1.2196667e+00 1.1994710e+00 1.1630678e+00 1.1155421e+00
 1.0276823e+00 9.9092388e-01 9.7901767e-01 1.0000000e+20 1.0000000e+20
 1.00

In [14]:
# tdir = '/data/tjarniko/results/BASERUN_EXP/MAIN/BR_2nd_2015/ncs/'
# gridnc = 'SKOG_*_grid_T_20150101-20150101.nc'
# carpnc = 'SKOG_*_carp_T_20150101-20150101.nc'

# tgrid = glob.glob(tdir+gridnc)
# print(tgrid)
# grid = nc.Dataset(tgrid[0])

# tcarp = glob.glob(tdir+carpnc)
# print(tcarp)
# carp = nc.Dataset(tcarp[0])

In [15]:
# start = time.time()
# pHr, OmAr = OmA_3D(grid,carp)
# end = time.time()

# print(end-start)

In [16]:
def make_nclen(start,end,ftype, sdir):
    fn_ar = []
    start_run = arrow.get(start)
    end_run = arrow.get(end)
    arrow_array = []
    for r in arrow.Arrow.span_range('day', start_run, end_run):
        arrow_array.append(r)

    dayslen = len(arrow_array)
    for i in range(0,dayslen):
        tdate = arrow_array[i][0]
        ddmmmyy = tdate.format('DDMMMYY').lower()
        ymd = tdate.format('YYYYMMDD')
        nc_sens = '/data/tjarniko/results/BASERUN_EXP/MAIN/' + sdir + '/ncs/SKOG_1d_*'+ ftype +'*' + ymd + '-' + ymd + '.nc'
        tnc_sens = glob.glob(nc_sens)
        fn_ar.append(tnc_sens[0])

    return fn_ar

def make_fname_ar(start,end,runtype):
    fn_ar = []
    start_run = arrow.get(start)
    end_run = arrow.get(end)
    arrow_array = []
    for r in arrow.Arrow.span_range('day', start_run, end_run):
        arrow_array.append(r)

    dayslen = len(arrow_array)
    for i in range(0,dayslen):
        tdate = arrow_array[i][0]
        ymd = tdate.format('YYYYMMDD')
        fname = runtype + '_OmA_pH_' + ymd +'.nc'
        fn_ar.append(fname)
    
    return fn_ar

def OmA_3D(grid,carp):
    tsal = grid.variables['vosaline'][0,:,:,:]
    ttemp = grid.variables['votemper'][0,:,:,:]
    tdic = carp.variables['dissolved_inorganic_carbon'][0,:,:,:]
    tta = carp.variables['total_alkalinity'][0,:,:,:]

    tsra = np.ravel(tsal)
    ttera = np.ravel(ttemp)
    ttara = np.ravel(tta) * 1e-3
    tdra = np.ravel(tdic) * 1e-3
    tzero = np.zeros_like(tsra)
    tpressure = np.zeros_like(tsra)
    tpressure[:] =1
    tzero = tpressure * 0 
        
    tsra_psu = tsra*35/35.16504
    ttera_is = gsw.t_from_CT(tsra,ttera,tzero)

    response_tup = mocsy.mvars(temp=ttera_is, sal=tsra_psu, alk=ttara, dic=tdra, 
                       sil=tzero, phos=tzero, patm=tpressure, depth=tzero, lat=tzero, 
                        optcon='mol/m3', optt='Tinsitu', optp='m',
                        optb = 'l10', optk1k2='m10', optkf = 'dg', optgas = 'Pinsitu')
    pH,pco2,fco2,co2,hco3,co3,OmegaA,OmegaC,BetaD,DENis,p,Tis = response_tup

    pHr = pH.reshape(40,898,398)
    OmAr = OmegaA.reshape(40,898,398)
    
    return pHr, OmAr

In [17]:

# w = nc.Dataset('/data/tjarniko/results/BASERUN_EXP/MAIN/OmA_pH_calculated/'+ 'BR_OmA_pH_20150101.nc')
# print(w['model_output']['OmAr'][0,250,250])




In [18]:

runtype = 'BR'
start = '2015-01-01'
end = '2015-12-31'

fn_ar = make_fname_ar(start,end,runtype)

BR_year_OmA = np.zeros([365,40,898,398])

for i in range(0,365):
    if i%20 == 0:
        print(i)
    t_nc = nc.Dataset('/data/tjarniko/results/BASERUN_EXP/MAIN/OmA_pH_calculated/' + fn_ar[i])
    BR_year_OmA[i,:,:,:] = t_nc['model_output']['OmAr'][:,:,:]

    
ncname = runtype + '_OmA.nc'
f = nc.Dataset(ncname,'w', format='NETCDF4') #'w' stands for write
g = f.createGroup('model_output')
g.createDimension('days', 365)
g.createDimension('depths', 40)
g.createDimension('ydir',898)
g.createDimension('xdir',398)

ts = g.createVariable('OmA','f4',('days','depths','ydir','xdir'))
ts[:] = BR_year_OmA
    
f.close()


0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360


In [19]:
runtype = 'PI'
start = '2015-01-01'
end = '2015-12-31'

fn_ar = make_fname_ar(start,end,runtype)

PI_year_OmA = np.zeros([365,40,898,398])

for i in range(0,365):
    if i%20 == 0:
        print(i)
    t_nc = nc.Dataset('/data/tjarniko/results/BASERUN_EXP/MAIN/OmA_pH_calculated/' + fn_ar[i])
    PI_year_OmA[i,:,:,:] = t_nc['model_output']['OmAr'][:,:,:]

    
ncname = runtype + '_OmA.nc'
f = nc.Dataset(ncname,'w', format='NETCDF4') #'w' stands for write
g = f.createGroup('model_output')
g.createDimension('days', 365)
g.createDimension('depths', 40)
g.createDimension('ydir',898)
g.createDimension('xdir',398)

ts = g.createVariable('OmA','f4',('days','depths','ydir','xdir'))
ts[:] = PI_year_OmA
    
f.close()

0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360


In [20]:
runtype = 'LA'
start = '2015-01-01'
end = '2015-12-31'

fn_ar = make_fname_ar(start,end,runtype)

LA_year_OmA = np.zeros([365,40,898,398])

for i in range(0,365):
    if i%20 == 0:
        print(i)
    t_nc = nc.Dataset('/data/tjarniko/results/BASERUN_EXP/MAIN/OmA_pH_calculated/' + fn_ar[i])
    LA_year_OmA[i,:,:,:] = t_nc['model_output']['OmAr'][:,:,:]

    
ncname = runtype + '_OmA.nc'
f = nc.Dataset(ncname,'w', format='NETCDF4') #'w' stands for write
g = f.createGroup('model_output')
g.createDimension('days', 365)
g.createDimension('depths', 40)
g.createDimension('ydir',898)
g.createDimension('xdir',398)

ts = g.createVariable('OmA','f4',('days','depths','ydir','xdir'))
ts[:] = LA_year_OmA
    
f.close()

0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360


In [21]:
def find_depth_deepalg(dp,prof,water_depth):
    #finds saturation horizon given a profile and corresponding depths
    first_proper_undersat = np.nan
    depth_undersat = np.nan    
    dummy_var = 0
    #print(prof)
    #print('')
    if np.ma.min(prof) >=1e19:
        dummy_var = 0
        depth_undersat = np.nan
        #print('masks all around!')
    elif np.ma.min(prof) >=1:
        depth_undersat = water_depth
        #print('saturated column')
    elif np.ma.max(prof) <1:
        depth_undersat = 0
        #print('undersat to surface')        
    else:
        t_ind = np.where(prof<1)
        t_indar = t_ind[0][0]
        t_indss = np.where(prof>=1)
        t_indsssar = t_indss[0][0]
        if t_indar.size == 0:
            dummy_var = 0
        else:
            if (t_indar.size != 0) & (t_indsssar.size == 0):
                depth_undersat = 0
                first_proper_undersat = 0
                dummy_var = 0
                #print('undersat to surface!')
                max_supsat = np.nan
            else:    
                max_supsat = np.max(t_indsssar)    
                try:
                    first_proper_undersat = np.min(t_indar[t_indar>max_supsat])
                except:
                    dummy_var = 0
                    #print("An exception occurred")
                if first_proper_undersat == 0:
                    depth_undersat = dp[0]
                elif np.isnan(first_proper_undersat):
                    dummy_var = 0
                    #print('saturated watercolumn!')
                else:
                    depth_undersat = (dp[first_proper_undersat]+dp[first_proper_undersat-1])/2
    return depth_undersat

def find_depth_shallowalg(dp,prof,water_depth):
    #finds saturation horizon given a profile and corresponding depths
    first_proper_undersat = np.nan
    depth_undersat = np.nan    
    dummy_var = 0
    #tot masked
    #print(prof)
    #print('')
    if np.ma.min(prof) >=1e19:
        dummy_var = 0
        depth_undersat = np.nan
        #print('masks all around!')
    elif np.ma.min(prof) >=1:
        depth_undersat = water_depth
        #print('saturated column')
    elif np.ma.max(prof) <1:
        depth_undersat = 0
        #print('undersat to surface')        
    else:
        t_ind = np.where(prof<1)
        t_indar = t_ind[0][0]
        t_indss = np.where(prof>=1)
        t_indsssar = t_indss[0][0]
        if t_indar.size == 0:
            dummy_var = 0
        else:
            if (t_indar.size != 0) & (t_indsssar.size == 0):
                depth_undersat = 0
                first_proper_undersat = 0
                dummy_var = 0
                #print('undersat to surface!')
                max_supsat = np.nan
            else:    
                max_supsat = np.max(t_indsssar)    
                try:
                    first_proper_undersat = np.min(t_indar)
                except:
                    dummy_var = 0
                    #print("An exception occurred")
                if first_proper_undersat == 0:
                    depth_undersat = dp[0]
                elif np.isnan(first_proper_undersat):
                    dummy_var = 0
                    #print('saturated watercolumn!')
                else:
                    depth_undersat = (dp[first_proper_undersat]+dp[first_proper_undersat-1])/2
    return depth_undersat

In [22]:
def find_sathor(j_st,j_en,tit,ncname,shallow):
    if shallow == False:
        print('using deep alg')
    if shallow == True:
        print('using shallow alg')
    OmA = nc.Dataset(ncname)
    BR_omA = OmA['model_output']['OmA']


    t_nc = nc.Dataset('/results2/SalishSea/nowcast-green.201812/01jan18/SalishSea_1h_20180101_20180101_grid_T.nc')
    bath =  nc.Dataset('/home/sallen/MEOPAR/grid/bathymetry_201702.nc')
    zlevels = (t_nc['deptht'][:])

    oma_d_BR = np.zeros([365,898,398])
    for day in range(0,365):
        print('day is ' + str(day))
        for j in range(j_st,j_en):
            if j%10 == 0:
                print('j is ' + str(j))
            for i in range(0,398):
                OmA_BR_test = BR_omA[day,:,j,i]
                
                water_depth = bath.variables['Bathymetry'][j,i]
                if shallow == False:
                    oma_dep_BR = find_depth_deepalg(zlevels,OmA_BR_test,water_depth)

                if shallow == True:
                    oma_dep_BR = find_depth_shallowalg(zlevels,OmA_BR_test,water_depth)
                oma_d_BR[day,j,i] = oma_dep_BR

                    
    ncname = tit + str(j_st) + '_' + str(j_en) + '.nc'
    f = nc.Dataset(ncname,'w', format='NETCDF4') #'w' stands for write
    g = f.createGroup('model_output')
#g.createDimension('days', len(3))
    g.createDimension('days', 365)
    g.createDimension('ys', 898)
    g.createDimension('xs', 398)
    ts = g.createVariable('OmAr_HORIZON','f4',('days','ys','xs'))
    ts[:] = oma_d_BR

    f.close()

In [23]:
# j_st = 0
# j_en = 100
# tit = 'BR_DEEPALG_'
# ncname = 'BR_OmA.nc'
# shallow = False
# find_sathor(j_st,j_en,tit,ncname,shallow)